In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_pickle('testingdf2.pkl')
df = df.replace(pd.NA, np.nan)
df

In [ ]:
# This cell removes all zeroes from the dataframe, if uncommented.

df = df.replace(np.nan, 0)
df = df.replace(0, np.nan)
df

In [ ]:
class MFRecommender():
    """
    Matrix factorization recommender model object.
    
    Attributes:
        game_weights (np.array): matrix of weights corresponding to user 
            profile weights for each game.
        user_weights (np.array): matrix of weights corresponding to how to 
            represent each user as a linear combination of user profiles.
        filled_entries (List[tuple(int)]): list of index pairs (i,j) of the
            non-null entries of df. 
        lr (float): learning rate
        l2 (float): size of the l2 penalty when fitting and predicting.
        df (pd.DataFrame): pandas dataframe representing the sparse matrix
            of data
    """
    
    
    def __init__(self, df, k=7, lr=0.0001, l2=0.1, val_split=0.15):
        """
        Initializes the model.
        
        Args:
            df (pd.DataFrame): sparse dataframe of user playtime info.
            k (int): Number of user profiles for the model.
            lr (float): learning rate for SGD.
            l2 (float): l2 penalty for training the weights.
        """
        self.df = df
        self.game_weights = np.random.rand(df.shape[0],k)
        self.user_weights = np.random.rand(k, df.shape[1])
        self.filled_entries = []
        self.lr = lr
        self.l2 = l2
        for i in range(df.shape[0]):
            for j in range(df.shape[1]):
                if not pd.isna(df.iat[i,j]):
                    self.filled_entries.append((i,j))
        self.fe_train, self.fe_valid = train_test_split(self.filled_entries, 
                                                       test_size=val_split)
        
    
    
    def fit(self, epochs):
        """
        Fits the model a certain number of epochs based on the df provided.
        Args:
            epochs (int): number of epochs to train the model
        """
        for k in range(epochs):
            print('Fitting epoch {}...'.format(k+1))
            pred = np.matmul(self.game_weights, self.user_weights)
            t_game_weights = np.copy(self.game_weights)
            t_user_weights = np.copy(self.user_weights)
            
            # apply l2 penalty
            self.game_weights = t_game_weights*(1 - self.lr*self.l2)
            self.user_weights = t_user_weights*(1 - self.lr*self.l2)
            
            # Update with SGD
            for i, j in self.fe_train:
                for l in range(self.user_weights.shape[0]):
                    diff = (self.df.iat[i,j]-pred[i,j])
                    self.game_weights[i,l] += self.lr*t_user_weights[l,j]*diff
                    self.user_weights[l,j] += self.lr*t_game_weights[i,l]*diff
            print('Train MSE = {:.5f}     '.format(self.train_MSE())
                 + 'Validation MSE = {:.5f}'.format(self.valid_MSE()))
    
    
    def train_loss(self):
        loss = 0
        pred = np.matmul(self.game_weights, self.user_weights)
        for i, j in self.fe_train:
            # print(self.df.iat[i,j], pred[i,j])
            loss += (self.df.iat[i,j] - pred[i,j])**2
        return loss
    
    def valid_loss(self):
        loss = 0
        pred2 = np.matmul(self.game_weights, self.user_weights)
        for i, j in self.fe_valid:
            # print(self.df.iat[i,j], pred[i,j])
            loss += (self.df.iat[i,j] - pred2[i,j])**2
        return loss
    
    def train_MSE(self):
        loss = 0
        pred2 = np.matmul(self.game_weights, self.user_weights)
        for i, j in self.fe_train:
            # print(self.df.iat[i,j], pred[i,j])
            loss += (self.df.iat[i,j] - pred2[i,j])**2
        return loss/len(self.fe_train)
    
    def valid_MSE(self):
        loss = 0
        pred2 = np.matmul(self.game_weights, self.user_weights)
        for i, j in self.fe_valid:
            # print(self.df.iat[i,j], pred[i,j])
            loss += (self.df.iat[i,j] - pred2[i,j])**2
        return loss/len(self.fe_valid)
    
    def predict(self, user_data, epochs=100):
        """
        Given the data for a user, returns the predicted playtime series.
        
        Args:
            user_data (pd.series): sparse pandas series of user playtime
            epochs (int): number of epochs to fit the user weights for
                the user who's user data was entered.
        
        Returns (pd.series): filled pandas series of predicted playtime.
        """
        user_profile = np.random.rand(self.user_weights.shape[0], 1)
        
        filled_indices = []
        for j in range(user_data.shape[0]):
            if not pd.isna(user_data.iat[j]):
                filled_indices.append(j)
        
        for j in range(epochs):
            #print('Fitting epoch {}...'.format(k+1))
            pred2 = np.matmul(self.game_weights, user_profile)
            #print(pred)
            t_user_weights = np.copy(user_profile)
            
            # apply l2 penalty
            user_profile = t_user_weights*(1 - self.lr*self.l2)
            
            # Update with SGD
            for i in filled_indices:
                for l in range(self.user_weights.shape[0]):
                    diff = (user_data.iat[i]-pred2[i])
                    user_profile[l] += self.lr*self.game_weights[i,l]*diff
        
        # return final prediction, converted to pandas
        to_return = user_data.copy()
        pred2 = np.matmul(self.game_weights, user_profile)
        for j in range(to_return.shape[0]):
            to_return.iat[j] = pred2[j]
            
        loss = 0
        for i in filled_indices:
            # print(self.df.iat[i,j], pred[i,j])
            loss += (user_data.iat[i] - pred2[i])**2
        print('MSE: {}'.format(loss/len(filled_indices)))
        return to_return
                
                
                
        

In [ ]:
# Note that the outliers in the data will prevent convergence unless we log-normalize first.

log_df = df.applymap(lambda x: np.log10(x+1) if not np.isnan(x) else x)
log_df

In [ ]:
clf = MFRecommender(log_df, k=10)
clf.fit(200)

In [ ]:
# validation MSE
clf.valid_MSE()

In [ ]:
# save optimal weights to a file so it can be reloaded later
np.save('./game_weights.npy', clf.game_weights)

In [ ]:
# Make a scatterplot of predicted values vs input values.
import matplotlib.pyplot as plt
def convert(x):
    return 10**x -1

tlist = [[], []]
pred = np.matmul(clf.game_weights, clf.user_weights)
for i, j in clf.fe_valid:
    tlist[0].append(clf.df.iat[i,j])
    tlist[1].append(pred[i, j])

plt.scatter(x=tlist[0], y= tlist[1], alpha=0.05)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('seaborn-v0_8')
# plt.style.use('bmh')
plt.hist(tlist, bins=100)
plt.title('Distribution of user playtimes')
plt.yscale('log')
plt.xlabel('playtime in min')
plt.ylabel('count')
plt.show()

In [ ]:
# Histogram of playtime values. Data seems to be log normally distributed.
dataframe = log_df
tlist = []
for i in range(len(clf.filled_entries)):
    j, k = clf.filled_entries[i]
    tlist.append(dataframe.iat[j, k])
    
plt.style.use('seaborn-v0_8')
# plt.style.use('bmh')
plt.hist(tlist, bins=100)
plt.title('Distribution of log user playtimes')
plt.xlabel('log_10(playtime) in min')
plt.ylabel('count')
plt.show()

In [ ]:
#try testing with a new user
from steam import Steam
from decouple import config
with open("myapikey.txt", "r") as file:
    myapikey=file.read()
    
import os
os.environ["STEAM_API_KEY"] = myapikey
KEY = config("STEAM_API_KEY")
steam = Steam(KEY)

gamenames=pd.read_pickle('gamenames.pkl')
gameprice=pd.read_pickle('gameprice.pkl')

pd.options.display.max_rows = 986

In [ ]:
gameprice2=[]

for price in gameprice:
    lenp=len(price)
    if len(price)>1:
        price2=price[:(lenp-2)]+'.'+price[(lenp-2):]
    else:
        price2=price
    price2=float(price2)
    gameprice2.append(price2)
    
gameprice2

In [ ]:
testid='76561198039393086'

In [ ]:
testsgamelist=steam.users.get_owned_games(testid)

In [ ]:
#make a column of user's gametime to use with trained model

testcol=pd.Series(np.nan,index=df.index)
for i in range(testsgamelist['game_count']):
    appid=testsgamelist['games'][i]['appid']
    if appid in df.index:
        if testsgamelist['games'][i]['playtime_forever']>0:
            testcol[appid]=testsgamelist['games'][i]['playtime_forever']

In [ ]:
testfit=clf.predict(np.log10(testcol+1))

In [ ]:
#makes a dataframe of the top 10 games by predicted playtime
results1=pd.DataFrame({'game_name':gamenames,'prediction (hours)':((10**testfit)-1)/60.})
results1.sort_values("prediction (hours)",ascending=False).head(10).round(1)

In [ ]:
#makes a dataframe of the lowest 10 games by game price/predicted playtime
results2=pd.DataFrame({'game_name':gamenames,'predicted cost/playtime (dollars/hour)':gameprice2/(((10**testfit)-1)/60)})
results2=results2.loc[results2['predicted cost/playtime (dollars/hour)']>0]
results2.sort_values("predicted cost/playtime (dollars/hour)").round(2).head(10)